### Using Keras with TensorFlow backend

In [1]:
import numpy as np

from keras.models import Model
from keras.layers import Flatten, Dense, Input, Activation
from keras.layers import BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing import image
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.models import Sequential
import h5py

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Set GPU
tf.device('/gpu:0') tells TF to use gpu 0. Can use gpu 1. If cell is not run then fills both GPUs

In [3]:
import tensorflow as tf
tf.device('/gpu:0')

In [2]:
#val = np.loadtxt('kaggle/val.txt',dtype='str',delimiter=' ')

In [3]:
#Grab image paths put in list
#path = 'kaggle/train_sm/'
#img_paths = []
#for i in val:
#    img_paths.append(path + i[0])
#print len(img_paths)

688


### Grab Test Image file paths and put into List

In [4]:
import os

In [5]:
#Grab image paths put in list
path = 'UCMerced/Images/'
img_paths = []
label = []
for i in sorted(os.listdir(path)):
    for j in sorted(os.listdir(path+i)):
        img_paths.append(path + i + '/'+ j)
        label.append(j)
print len(img_paths)

2100


In [4]:
new_path = 'Cifar/Val/'
#Grab image paths put in list
img_paths = []
for i in sorted(os.listdir(new_path)):
    small_path = new_path+i+'/'
    for j in sorted(os.listdir(small_path)):
        img_paths.append(small_path + j)
print len(img_paths)

10000


### Import Functional Keras model from keras.applications Library 

In [6]:
#from keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from myresnet50 import ResNet50

### Create Instance of base model 
include_top = False gets rid of top model.

weights = 'imagenet' Sets weights to image net weights.

In [7]:
model = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))

#model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))


K.image_dim_ordering: tf


### Build Top Model

In [8]:
# ResNet Top Model
x = model.output
x = Flatten(name='flatten')(x) 
x = BatchNormalization(axis=1, name='batch_norm')(x)

# 344 is the number of classes
x = Dense(344, name='fc1000')(x)
x = Activation("softmax",name='bfc1000')(x)


### Create a New Model Class where input is base model and output is Top model

In [9]:
# Stack top_model on top
final_model = Model(input=model.input, output=x)

In [10]:
#File handle to open weights path
weights_path = 'Caffe models/ResNet50/weights/ResNet50-finetune5-14-0.66.h5'
#weights_path = 'Caffe models/VGG/VGG16-finetune2-24-0.68.h5'
f = h5py.File(weights_path)

In [14]:
#K.get_session().run(tf.global_variables_initializer())

### For each layer set the weights from the .h5 file

In [11]:
for k in range(len(f.attrs['layer_names'])): 
    name = f.attrs['layer_names'][k]
    weights = [f[name][h][()] for h in f[name].keys()]
    print k
    final_model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
Model loaded.


### Create a new model to feature extract and any desired layer

In [16]:
layer_name = 'avg_pool'
intermediate_layer_model = Model(input=final_model.input,
                                 output=final_model.get_layer(layer_name).output)
#intermediate_layer_model = Model(input=model.input,
#                                 output=model.get_layer('avg_pool').output)

In [17]:
def preprocess_input(x):
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    x = x*(1./255)
    return x

In [18]:
#Test for 1 image 
img = image.load_img(img_paths[0], target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
test = intermediate_layer_model.predict(x)

In [1]:
test

In [ ]:
#Get the first weight of each layer to check how big by sending image through
for name in f.attrs['layer_names']:
    try:
        model.get_layer(name).output
    except:
        continue
    intermediate_layer_model = Model(input=model.input,
                                     output=model.get_layer(name).output)
    img = image.load_img(img_paths[0], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test = intermediate_layer_model.predict(x)
    print name
    print test[0].flatten()[0]


In [20]:
features = []
for i in range(len(img_paths)):
    img = image.load_img(img_paths[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(intermediate_layer_model.predict(x).flatten())

In [21]:
image_features = np.squeeze(features)

In [22]:
image_features.shape

(2100, 2048)

### Process to Save extracted Features into Dataframe then to CSV

In [23]:
import pandas as pd

In [24]:
df1 = pd.DataFrame(np.array(label))
df2 = pd.DataFrame(image_features)

In [25]:
frames = [df1, df2]
result = pd.concat(frames,axis=1)

In [26]:
result.head()

,0,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,agricultural00.tif,8.001358,6.153831,6.254327,10.351535,6.470306,11.839881,7.360807,9.744673,7.371373,...,8.543665,9.858485,7.902946,8.033251,8.304284,9.314300,6.383990,8.188457,8.081923,7.256083
1,agricultural01.tif,8.001143,6.153898,6.254155,10.351558,6.470414,11.839757,7.360776,9.744571,7.371409,...,8.543505,9.858540,7.902856,8.033365,8.304240,9.314152,6.383946,8.188575,8.081857,7.255932
2,agricultural02.tif,8.001007,6.153821,6.254086,10.351634,6.470341,11.839730,7.360781,9.744671,7.371364,...,8.543264,9.858466,7.903122,8.033354,8.304253,9.314267,6.383912,8.188551,8.081882,7.255896
3,agricultural03.tif,8.001159,6.153828,6.254199,10.351539,6.470387,11.839847,7.360753,9.744618,7.371356,...,8.543521,9.858546,7.902995,8.033306,8.304213,9.314361,6.383863,8.188568,8.081996,7.255991
4,agricultural04.tif,8.003878,6.154348,6.253857,10.351818,6.469512,11.839909,7.361197,9.745896,7.369430,...,8.543104,9.857647,7.903005,8.032755,8.304073,9.312605,6.385001,8.189698,8.080696,7.256610


In [27]:
result.to_csv('Features Extracted/UCMerced/Finetune/Single/UCM-ResNet50-14epoch-avgpool-66-fine5.csv',index=False)